In [3]:
import pandas as pd
import _sqlite3

df = pd.read_csv("nba_salaries_clean.csv")
df["season"] = "2022-23"

In [4]:
conn = _sqlite3.connect("nba_salaries.db")
cursor = cursor = conn.cursor()

In [5]:
teams = df["team"].drop_duplicates()

for team in teams:
    cursor.execute(
        "INSERT INTO teams (team_name) VALUES (?)",
        (team,)
    )


In [6]:
players = df[["player_name", "team"]].drop_duplicates()

for _, row in players.iterrows():
    cursor.execute(
        """
        INSERT INTO players (player_name, team_id)
        SELECT ?, team_id
        FROM teams
        WHERE team_name = ?
        """,
        (row["player_name"], row["team"])
    )


In [7]:
for _, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO salaries (player_id, season, salary_usd)
        SELECT player_id, ?, ?
        FROM players
        WHERE player_name = ?
        """,
        (row["season"], int(row["salary"]), row["player_name"])
    )


In [8]:
conn.commit()
conn.close()

In [9]:
print("CSV data successfully inserted into SQLite database")


CSV data successfully inserted into SQLite database
